In [27]:
from sys import path
from constants import PROJECT_DIR
path.insert(1, PROJECT_DIR)

from master.manager.stage_manager import StageManager
from base_utils import path_join
from constants import DEFAULT_PARAM_TEMPLATE, MASTER_DATABASE_DIR

import pandas as pd

In [44]:
def stage_stats(stage):
    stage_manager = StageManager(DEFAULT_PARAM_TEMPLATE)
    stage_info = stage_manager.get_stage_info(stage)
    eval_results = stage_info["eval_results"]
    winrates = {model_id:[eval_results[model_id][opp_id]["winrate"] for opp_id in eval_results[model_id]] for model_id in eval_results}
    model_ids = list(winrates.keys())
    avg_winrates = [sum(winrates[model_id])/len(winrates[model_id]) for model_id in winrates]
    parent, policy, learning_rate, gamma, gae_lambda = zip(*read_models_info(stage, model_ids))
    data = zip(model_ids, avg_winrates, parent, policy, learning_rate, gamma, gae_lambda)
    scores_df = pd.DataFrame(data, columns=["model_id", "score", "parent", "policy", "lr", "gamma", "gae_lambda"])
    return scores_df

def read_models_info(stage, models, database=MASTER_DATABASE_DIR):
    data = []
    for model in models:
        model_path = path_join(database, stage, model)
        with open(path_join(model_path, "info.json")) as f:
            model_info = eval(f.read())
        parent = model_info["parents"][-1] if len(model_info["parents"]) > 0 else None
        data.append([parent, *model_info["train_params"].values()])
    return data

def stage_best_stats(stage):
    stage_manager = StageManager(DEFAULT_PARAM_TEMPLATE)
    stage_info = stage_manager.get_stage_info(stage)
    scores_df = stage_stats(stage)
    best_models = stage_info["best_models"]
    mask = [model_id in best_models for model_id in scores_df["model_id"]]
    return scores_df.loc[mask].sort_values("score", ascending=False).reset_index(drop=True)

In [51]:
stages = [f"stage_{i}" for i in range(1, 12)]

def get_all_best_stats(stages):
    df = pd.DataFrame([], columns=["model_id", "score", "parent", "policy", "lr", "gamma", "gae_lambda", "stage"])
    for stage in stages:
        stage_df = stage_best_stats(stage)
        stage_df["stage"] = stage
        df = df.append(stage_df).reset_index(drop=True)

    df.reset_index(drop=True)
    return df

,model_id,score,parent,policy,lr,gamma,gae_lambda,stage
0,27495,0.73,62633,agent1,0.0010,0.995,0.95,stage_1
1,55448,0.73,69879,agent1,0.0004,0.995,0.95,stage_1
2,67656,0.68,22598,agent2,0.0004,0.995,0.95,stage_1
3,70373,0.65,69879,agent1,0.0004,0.995,0.95,stage_1
4,76561,0.65,69879,agent1,0.0004,0.995,0.95,stage_1
...,...,...,...,...,...,...,...,...
77,97441,0.58,91872,agent3,0.0004,0.995,0.95,stage_11
78,99439,0.56,99823,agent3,0.0001,0.995,0.95,stage_11
79,73523,0.53,66600,agent2,0.0001,0.995,0.95,stage_11
80,56281,0.51,66600,agent2,0.0001,0.995,0.95,stage_11
